In [69]:
import pandas as pd
import numpy as np
import re

In [83]:
#guitars = pd.read_csv('../data/all_used_guitars.csv')
guitars = pd.read_csv('../data/partial_df.csv')
guitars.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21108 entries, 0 to 21107
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  21108 non-null  int64 
 1   title       21108 non-null  object
 2   price       21108 non-null  object
 3   condition   21108 non-null  object
 4   url         21108 non-null  object
dtypes: int64(1), object(4)
memory usage: 824.7+ KB


In [84]:
guitars.head()

,Unnamed: 0,title,price,condition,url
0,0,2017 Music Man Stingray Chili Red Electric Gui...,$59,Used – Very Good,https://reverb.com/item/50609689-2017-music-ma...
1,1,Jay Turser Dragon Inlay 2000s Red Burst,$500,Used – Good,https://reverb.com/item/50480779-jay-turser-dr...
2,2,B.C. Rich Warlock Acrylic 1999 Translucent Red,"$1,300",Used – Excellent,https://reverb.com/item/50356042-b-c-rich-warl...
3,3,Sterling Jp160 Black metallic,$850,Used – Very Good,https://reverb.com/item/50609683-sterling-jp16...
4,4,G&L Legacy 2015 Red burst,"$1,275",Used – Mint,https://reverb.com/item/50555692-g-l-legacy-20...


In [85]:
brands = {
       'Airline':    'Airline',
       'Alvarez':    'Alvarez',
       'American Showster':    'American Showster',
       'Ampeg':    'Ampeg',
       'Aria':    'Aria',
       'B.C. Rich':    'B.C. Rich',
       'B3':    'B3',
       'BC Rich':    'BC Rich',
       'Baldwin':    'Baldwin',
       'Bilt':    'Bilt',
       'Bogner':    'Bogner',
       'Burns':    'Burns',
       'Campbell':    'Campbell',
       'Charvel':    'Charvel',
       'Collings':    'Collings',
       'Conrad':    'Conrad',
       'Cort':    'Cort',
       "D'Angelico":    "D'Angelico",
       'Danelectro':    'Danelectro',
       'DeArmond':    'DeArmond',
       'Dean':    'Dean',
       'DiPinto':    'DiPinto',
       'Dobro':    'Dobro',
       'Duesenberg':    'Duesenberg',
       'EKO':    'EKO',
       'ESP':    'ESP',
       'EVH':    'EVH',
       'Eastman':    'Eastman',
       'Eastwood':    'Eastwood',
       'Electra':    'Electra',
       'Epiphone':    'Epiphone',
       'Ernie Ball Music Man':    'Ernie Ball Music Man',
       'Fano':    'Fano',
       'Fender':    'Fender',
       'FENDER':    'Fender',        
       'Framus':    'Framus',
       'G&L':    'G&L',
       'Gagliano':    'Gagliano',
       'Gibson':    'Gibson',
        'GIBSON':    'Gibson',
       'Giffin':    'Giffin',
       'Godin':    'Godin',
       'Goya':    'Goya',
       'Gretsch':    'Gretsch',
       'Grosh':    'Grosh',
       'Guild':    'Guild',
       'Guyatone':    'Guyatone',
       'Hagstrom':    'Hagstrom',
       'Hamer':    'Hamer',
       'Harden Engineering':    'Harden Engineering',
       'Harmony':    'Harmony',
       'Heritage':    'Heritage',
       'Harley Benton':    'Harley Benton',
       'Höfner':    'Höfner',
       'Hohner':    'Höfner',
       'Ibanez':    'Ibanez',
       'Intermark':    'Intermark',
       'Jackson':    'Jackson',
       'James Trussart':    'James Trussart',
       'Jerry Jones':    'Jerry Jones',
       'Kalamazoo':    'Kalamazoo',
       'Kay':    'Kay',
       'Kimberly':    'Kimberly',
       'Kramer':    'Kramer',
       'LTD':    'ESP',        
       'La Baye':    'La Baye',
       'Larrivee':    'Larrivee',
       'Line 6':    'Line 6',
       'MCI':    'MCI',
       'Michael Kelly':'Michael Kelly',
       'Magnatone':    'Magnatone',
       'Martin':    'Martin',
       'Memphis':    'Memphis',
       'Micro-Frets':    'Micro-Frets',
       'Moog':    'Moog',
       'Mosrite':    'Mosrite',
       'Music Man':    'Music Man',
       'Musicraft':    'Musicraft',
       'Musicvox':    'Musicvox',
       'Nash':    'Nash',
       'National':    'National',
       'Norma':    'Norma',
       'Orville':    'Orville',
       'Ovation':    'Ovation',
       'Parker':    'Parker',
       'PRS':    'PRS',
       'Peavey':    'Peavey',
       'Premier':    'Premier',
       'Reverend':    'Reverend',
       'Rickenbacker':    'Rickenbacker',
       'Roland':    'Roland',
       'SWR':    'SWR',
       'Schecter':    'Schecter',
       'Silvertone':    'Silvertone',
       'Sorrento':    'Sorrento',
       'Squier':    'Squier',
       'Steinberger':    'Steinberger',
       'Suhr':    'Suhr',
       'Supro':    'Supro',
       'Strandberg':    'Strandberg',
       'Traveler':    'Traveler',        
       'Taylor':    'Taylor',
       'Teisco':    'Teisco',
       'The Loar':    'The Loar',
       'Tokai':    'Tokai',
       'Tom Anderson':    'Tom Anderson',
       'Travis':    'Travis',
       'Truetone':    'Truetone',
       'Two Rock':    'Two Rock',
       'Univox':    'Univox',
       'Vox':    'Vox',
       'Washburn':    'Washburn',
       'Warmoth':    'Warmoth',
       'Woodrow':    'Woodrow', 
       'Yamaha':    'Yamaha'   }

def clean_name(df):
    pattern = '|'.join(brands)
    df['state'] = df.title.str.contains(pattern)
    
    
    

def extractbrands(df,brands):
    brands = {
           'Airline':    'Airline',
           'Alvarez':    'Alvarez',
           'American Showster':    'American Showster',
           'Ampeg':    'Ampeg',
           'Aria':    'Aria',
           'B.C. Rich':    'B.C. Rich',
           'B3':    'B3',
           'BC Rich':    'BC Rich',
           'Baldwin':    'Baldwin',
           'Bilt':    'Bilt',
           'Bogner':    'Bogner',
           'Burns':    'Burns',
           'Campbell':    'Campbell',
           'Charvel':    'Charvel',
           'Collings':    'Collings',
           'Conrad':    'Conrad',
           'Cort':    'Cort',
           "D'Angelico":    "D'Angelico",
           'Danelectro':    'Danelectro',
           'DeArmond':    'DeArmond',
           'Dean':    'Dean',
           'DiPinto':    'DiPinto',
           'Dobro':    'Dobro',
           'Duesenberg':    'Duesenberg',
           'EKO':    'EKO',
           'ESP':    'ESP',
           'EVH':    'EVH',
           'Eastman':    'Eastman',
           'Eastwood':    'Eastwood',
           'Electra':    'Electra',
           'Epiphone':    'Epiphone',
           'Ernie Ball Music Man':    'Ernie Ball Music Man',
           'Fano':    'Fano',
           'Fender':    'Fender',
           'FENDER':    'Fender',        
           'Framus':    'Framus',
           'G&L':    'G&L',
           'Gagliano':    'Gagliano',
           'Gibson':    'Gibson',
           'GIBSON':    'Gibson',
           'Giffin':    'Giffin',
           'Godin':    'Godin',
           'Goya':    'Goya',
           'Gretsch':    'Gretsch',
           'Grosh':    'Grosh',
           'Guild':    'Guild',
           'Guyatone':    'Guyatone',
           'Hagstrom':    'Hagstrom',
           'Hamer':    'Hamer',
           'Harden Engineering':    'Harden Engineering',
           'Harmony':    'Harmony',
           'Heritage':    'Heritage',
           'Harley Benton':    'Harley Benton',
           'Höfner':    'Höfner',
           'Hohner':    'Höfner',
           'Ibanez':    'Ibanez',
           'Intermark':    'Intermark',
           'Jackson':    'Jackson',
           'James Trussart':    'James Trussart',
           'Jerry Jones':    'Jerry Jones',
           'Kalamazoo':    'Kalamazoo',
           'Kay':    'Kay',
           'Kimberly':    'Kimberly',
           'Kramer':    'Kramer',
           'LTD':    'ESP',        
           'La Baye':    'La Baye',
           'Larrivee':    'Larrivee',
           'Line 6':    'Line 6',
           'MCI':    'MCI',
           'Michael Kelly':'Michael Kelly',
           'Magnatone':    'Magnatone',
           'Martin':    'Martin',
           'Memphis':    'Memphis',
           'Micro-Frets':    'Micro-Frets',
           'Moog':    'Moog',
           'Mosrite':    'Mosrite',
           'Music Man':    'Music Man',
           'Musicraft':    'Musicraft',
           'Musicvox':    'Musicvox',
           'Nash':    'Nash',
           'National':    'National',
           'Norma':    'Norma',
           'Orville':    'Orville',
           'Ovation':    'Ovation',
           'Parker':    'Parker',
           'PRS':    'PRS',
           'Peavey':    'Peavey',
           'Premier':    'Premier',
           'Reverend':    'Reverend',
           'Rickenbacker':    'Rickenbacker',
           'Roland':    'Roland',
           'SWR':    'SWR',
           'Schecter':    'Schecter',
           'Silvertone':    'Silvertone',
           'Sorrento':    'Sorrento',
           'Squier':    'Squier',
           'Steinberger':    'Steinberger',
           'Suhr':    'Suhr',
           'Supro':    'Supro',
           'Strandberg':    'Strandberg',
           'Traveler':    'Traveler',        
           'Taylor':    'Taylor',
           'Teisco':    'Teisco',
           'The Loar':    'The Loar',
           'Tokai':    'Tokai',
           'Tom Anderson':    'Tom Anderson',
           'Travis':    'Travis',
           'Truetone':    'Truetone',
           'Two Rock':    'Two Rock',
           'Univox':    'Univox',
           'Vox':    'Vox',
           'Washburn':    'Washburn',
           'Warmoth':    'Warmoth',
           'Woodrow':    'Woodrow', 
           'Yamaha':    'Yamaha'   }    
        
        
    df['brand'] = (
        df.title
          .str.replace('Paul Reed Smith','PRS')
          .str.extract('(' + '|'.join(brands.keys()) + ')')
          .squeeze().map(brands)
    )
    
def extractmodel(df):
    model_dict = {
        '173':'173',
        '330':'330',
        '1454 Archtop':'1454 Archtop',
        'Artcore AGS73T':'Artcore AGS73T',
        'AT-200':'AT-200',
        'AT100CL-SB':'AT100CL-SB',
        'AZ':'AZ',
        'Bird of Prey': 'Bird of Prey',
        'Boden':'Boden',
        'Brad Paisley Esquire':'Brad Paisley Esquire',
        'C-1':'C-1',
        'C-7':'C-7',
        'Casino':'Casino',
        'Classic 1478':'Classic 1478',
        'Conqueror / Teisco / Matsumoku':'special',
        'Teisco / Kingston / Matsumoku "Surfcaster" ':'Surfcaster',
        'Classic 1423':'Classic 1423',
        'Cutlas':'Cutlas',
        'Contender':'Contender',
        'Coronado':'Coronado',
        'Cyclone':'Cyclone',
        'Dana':'Dana',
        'DELUXE DC LE':'DELUXE DC LE',
        "D'Angelico Deluxe SS":'DELUXE SS LE',
        'Deluxe Bedford':'Deluxe Bedford',
        'Dead-On':'Dead-On',
        'Dean V':'V',
        'Diablo':'Diablo',
        'Double Cat': 'Double Cat',
        "D'Angelico 6 String Semi-Hollow-Body":'HB',
        'G&L USA Doheny Hardtail':'Doheny Hardtail',
        'EC1000ETFM':'EC1000ETFM',
        'Epiphone SG Standard':'SG Standard',
        'Epiphone SG Classic':'SG Classic',
        'Epiphone SG Traditional':'SG Traditional',
        'Epiphone SG':'SG',
        'ES-325':'ES-325',
        'ES-339':'ES-339',
        'ESP LTD Snakebyte':'Snakebyte',
        'EG-1':'EG-1',
        'Fender Ben Gibbard Mustang':'Mustang',
        'ESP USA M-I':'M-I',
        'GPS':'GPS',
        'G5410T':'G5410T',
        'Guyatone Zenon Teisco':'generic/clone',
        'G6129T':'G6129T',
        'GRX70QA':'GRX70QA',
        'Gristlemaste':'Gristlemaster',
        'Electromatic':'Electromatic',
        'T486-GB':'T486-GB',
        'Falcon':'Falcon',
        'Harley Benton CST':'CST',
        'Harmony Marquis':'Marquis',
        'H78':'H78',
        'HT7':'HT7',
        'H-1001':'H-1001',
        'H-1001FR':'H-1001FR',
        'Horizon':'Horizon',
        'Idolmaker':'Idolmaker',
        'Impact 2':'Impact 2',
        'Jaguar':'Jaguar',
        'JAG':'Jaguar',
        'JAGUAR':'Jaguar',
        'Jazzmaster':'Jazzmaster',
        'Jersey Star':'Jersey Star',
        'Joe Satriani':'Joe Satriani',
        'John Petrucci Majesty':'Majesty',
        'JS Series Dinky™':'JS Dinky',
        'Kirk Hammett Signature':'Kirk Hammett Signature',
        'LP':'Les Paul',
        'Les Paul Standard':'Les Paul Standard',
        'Les Paul Classic':'Les Paul Classic',
        'Les Paul Studio':'Les Paul Studio',
        'Les Paul Tribute':'Les Paul Tribute',
        'Les Paul Custom':'Les Paul Custom',
        'Les Paul Special':'Les Paul Special',
        'Les Paul Junior':'Les Paul Junior',
        'Les Paul Traditional':'Les Paul Traditional',
        'LH-309':'LH-309',
        'LH-304T':'LH-304T',
        'Mod-X':'Mod-X',
        'M-I':'M-I',
        'M-200':'M-200',
        'MH200':'MH200',
        'Michael Kelly Mod Shop':'Mod Shop',
        'Michael Kelly Forte':'Forte', 
        'Michael Kelly Mod 55':'Mod Shop 55',
        'Mod Shop 50':'Mod Shop 50',
        'Mockingbird':'Mockingbird',
        'Monarkh':'Monarkh',
        'Omen':'Omen',
        'Parker Fly':'Fly',
        'Panthera Supreme':'Panthera Supreme',
        'Player Mustang':'Player Mustang',
        'Player Stratocaster':'Player Stratocaster',
        'Player Telecaster':'Player Telecaster',
        'Player Jaguar':'Player Jaguar',
        'Player Jazzmaster':'Player Jazzmaster',
        'PRS SE':'SE',
        'PRS S2':'S2',
        'PRS Custom 24':'PRS Custom 24',
        'PRS McCarty SC594':'McCarty SC594',
        'Reverse Explorer':'Reverse Explorer',
        'San Dimas Style 1':'San Dimas Style 1',
        'SG Standard':'SG Standard',
        'SG Classic':'SG Classic',
        'SG Traditional':'SG Traditional',
        'SG Junior':'SG Junior',
        'Trad Pro':'Trad Pro',
        'SA-500':'SA-500',
        'S-63':'S-63',
        'Special 22':'Special 22',
        'strat':'Stratocaster',
        'Strat':'Stratocaster',
        'STRATOCASTER':'Stratocaster',
        'Sheraton II':'Sheraton II',
        'Sheraton':'Sheraton',
        'Schecter Sun Valley':'Sun Valley',
        'Stingray':'Stingray',
        'Schecter 3327':'3327',
        'Streamliner':'Streamliner',
        'Soloist':'Soloist',
        'So-Cal Style 2':'So-Cal Style 2',
        'Suhr Standard':'Suhr Standard',
        'Tele':'Telecaster',
        'tele':'Telecaster',
        'T486B':'T486B',
        'Artist 2618':'Artist 2618',
        'ES-335': 'ES-335',
        'EC-1000':'EC-1000',
        'ET-200':'ET-200',
        'EXL-1':'EXL-1',
        'Northender':'Northender',
        'RG':'RG',
        'Silver Sky':'Silver Sky',
        'Tom Quayle Signature':'TQM',
        'Valentine':'Valentine',
        'Wolfgang Special':'Wolfgang Special',
        'Wood Library Custom 24':'Wood Library Custom 24',
        'Wildkat':'Wildkat',
        'X Series':'X Series'
    
    }
    df['model'] = (
        df.title
          .str.extract('(' + '|'.join(model_dict.keys()) + ')')
          .squeeze().map(model_dict)
    )
        
        
    

def cleantext(title):
        title = title.replace('(Mostly Loaded) Must See!','')
        title = title.replace('Pre-owned','')
        title = title.replace('Paul Reed Smith','PRS')
        title = title.replace('In Stock!','')
        title = title.replace('AUTUMN OFFLOAD// RARE ','')
        
        
         
       
        return title

def clean_price(txt):
    txt = txt.replace('$','')
    txt = txt.replace(',','')
    #txt = "{:.2f}".format(float(txt))
    #txt = float(txt)
    return float(txt)

def extractyear(title):
    pass

In [86]:
guitars['title'] = guitars.title.apply(cleantext)
guitars['price'] = guitars.price.apply(clean_price)

extractbrands(guitars,brands)
extractmodel(guitars)
#guitars[guitars.model.isna()].tail()

In [87]:
guitars.head()

,Unnamed: 0,title,price,condition,url,brand,model
0,0,2017 Music Man Stingray Chili Red Electric Gui...,59.0,Used – Very Good,https://reverb.com/item/50609689-2017-music-ma...,Music Man,Stingray
1,1,Jay Turser Dragon Inlay 2000s Red Burst,500.0,Used – Good,https://reverb.com/item/50480779-jay-turser-dr...,NaN,NaN
2,2,B.C. Rich Warlock Acrylic 1999 Translucent Red,1300.0,Used – Excellent,https://reverb.com/item/50356042-b-c-rich-warl...,B.C. Rich,NaN
3,3,Sterling Jp160 Black metallic,850.0,Used – Very Good,https://reverb.com/item/50609683-sterling-jp16...,NaN,NaN
4,4,G&L Legacy 2015 Red burst,1275.0,Used – Mint,https://reverb.com/item/50555692-g-l-legacy-20...,G&L,NaN


In [88]:
## drop non-branded
guitars = guitars.dropna(subset=['model'])

In [89]:
guitars['title'] = guitars.title.apply(cleantext)
extractbrands(guitars,brands)
extractmodel(guitars)
guitars[guitars.model.isna()].tail()

,Unnamed: 0,title,price,condition,url,brand,model


In [78]:
guitars.model.isna().sum()

0

In [90]:
guitars.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 9718 entries, 0 to 21107
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  9718 non-null   int64  
 1   title       9718 non-null   object 
 2   price       9718 non-null   float64
 3   condition   9718 non-null   object 
 4   url         9718 non-null   object 
 5   brand       8986 non-null   object 
 6   model       9718 non-null   object 
dtypes: float64(1), int64(1), object(5)
memory usage: 607.4+ KB


In [91]:
guitars.model.value_counts()

Stratocaster         3060
Telecaster           1580
Les Paul Standard     520
RG                    386
Les Paul Custom       341
                     ... 
Mod Shop                1
SG Traditional          1
GPS                     1
LH-309                  1
Double Cat              1
Name: model, Length: 104, dtype: int64

In [92]:
guitars.model.nunique()

104

In [81]:
guitars.to_csv('../data/guitars02102022.csv', encoding='utf-8')

In [43]:
guitars

,Unnamed: 0,title,price,condition,url,brand,model
0,0,Warmoth Stratocaster,$950,Used – Very Good,https://reverb.com/item/45652241-warmoth-strat...,Warmoth,Stratocaster
1,1,Fender American Professional II Stratocaster E...,"$1,599.99",Brand New,https://reverb.com/item/37165929-fender-americ...,Fender,Stratocaster
2,2,"Eastman T486B-GB Gold Burst Deluxe 16"" Thinlin...","$1,319",Brand New,https://reverb.com/item/45652293-eastman-t486b...,Eastman,T486B
3,3,Ibanez Artist 2618/12 string 1979 antique violin,"$2,499",Used – Very Good,https://reverb.com/item/45652250-ibanez-artist...,Ibanez,Artist 2618
4,4,Pre-Owned 1988 Fender Made In Japan Stratocast...,"$1,299.99",Used – Very Good,https://reverb.com/item/45402369-pre-owned-198...,Fender,Stratocaster
...,...,...,...,...,...,...,...
111298,111298,Fender Wildwood Coronado II (1967),"$3,150",Used – Excellent,https://reverb.com/item/45653703-fender-wildwo...,Fender,Coronado
111299,111299,"Squier (450) Bullet® Stratocaster® HT HSS, Lau...",$189.99,Brand New,https://reverb.com/item/45653624-squier-450-bu...,Squier,Stratocaster
111300,111300,2016 Squier Classic Vibes 50s Stratocaster She...,$699.98,Used – Excellent,https://reverb.com/item/45653622-2016-squier-c...,Squier,Stratocaster
111301,111301,Reverend Contender HB Metallic Alpine,$902,Used – Mint,https://reverb.com/item/45653603-reverend-cont...,Reverend,Contender
